# All PyZX Features

## Contents:
* [Loading and saving circuits](#circuits)
* [Interacting with Quantomatic](#quantomatic)
* [Optimizing ZX-diagrams](#optimization-zx)
* [Extracting and optimizing circuits](#optimization-circuits)

In [ ]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

<a id="circuits"></a>
# Loading and saving circuits
The most straightforward way to load a circuit is to use ``zx.Circuit.load`` which tries to figure out in which file format the circuit is given. The supported file formats are QASM, QC and the Quipper ASCII format. The corresponding loading functions are:
* ``zx.Circuit.from_qasm_file``
* ``zx.Circuit.from_quipper_file``
* ``zx.Circuit.from_qc_file``

In [ ]:
fname = os.path.join('..','circuits','Fast', 'mod5_4_before')
circ = zx.Circuit.load(fname)
# Alternatively we could have done:
# circ = zx.Circuit.from_quipper_file(fname)
zx.draw(circ, figsize=(10,2), h_edge_draw='box')

We can ask for simple statistics for this circuit:

In [ ]:
print(circ.stats())

This does not give very accurate information as the majority of this circuit is contained inside of CCZ gates. We can write the circuit in terms of basic gates to get more accurate statistics:

In [ ]:
print(circ.to_basic_gates().stats())

A Circuit can be exported into the supported file formats using
    * ``circ.to_qasm``
    * ``circ.to_quipper``
    * ``circ.to_qc``

In [ ]:
print(circ.to_qasm())

In [ ]:
print(circ.to_quipper())

In [ ]:
print(circ.to_qc())

<a id="quantomatic"></a>
# Interacting with Quantomatic
PyZX allows easy integration with quantomatic.

First of all, Quantomatic graph files can be imported into PyZX:

In [ ]:
fname = os.path.join('..','circuits','2-qubit-unitary.qgraph')
with open(fname, 'r') as f:
    g = zx.json_to_graph(f.read())
zx.draw(g, labels=True)

PyZX saves the names of the vertices:

In [ ]:
print(g.vdata(12,'name'))
print(g.vdata(1,'name'))

Because this graph was originally exported from PyZX, it has automatically remembered what its inputs and outputs are:

In [ ]:
g.inputs, g.outputs

For a graph that originated from Quantomatic we need to tell it what its inputs and outputs are.

This can be done either manually:

    ``g.inputs = list_of_inputs; g.outputs = list_of_outputs``
Or by PyZX trough calling ``g.auto_detect_inputs()``. This function makes all the vertices on the leftmost row inputs, and the vertices on the rightmost row outputs. For example:

In [ ]:
g.inputs = []
g.outputs = []
g.auto_detect_inputs()
print(g.inputs, g.outputs)

We can also call Quantomatic from PyZX. To do this we first need to tell PyZX where the Quantomatic executable can be found:

In [ ]:
zx.quantomatic.quantomatic_location = os.path.join('path', 'to', 'Quantomatic.jar')

Now, we can load a PyZX graph into Quantomatic using the following line:

In [ ]:
result = zx.quantomatic.edit_graph(g)

This starts Quantomatic with the graph ``g`` loaded. When you are done editing the graph, you simply save the file in Quantomatic, and close it. The result is then loaded and returned.

NOTE1: The Notebook will be blocked until the Quantomatic executable is closed.

NOTE2: Currently this only works with a recent build of Quantomatic that is as of yet only available via the repository, so make sure you are working with an up-to-date branch of Quantomatic.

<a id="optimization-zx"></a>
# Optimizing ZX-diagrams
PyZX contains many functions for optimizing circuits and ZX-diagrams. In this section we will show how these methods work and can be called.

First, let us load a small circuit:

In [ ]:
fname = os.path.join('..','circuits','Fast', 'mod5_4_before')
circ = zx.Circuit.load(fname)
print("original T-count:", zx.tcount(circ))
zx.draw(circ, figsize=(10,2))

The most basic simplification routine for ZX-graphs is ``interior_clifford_simp``. This uses the simplification rules based on spider-fusion, identity removal, pivoting and local complementation until they cannot be applied anymore.

In [ ]:
g = circ.to_graph() # We first have to convert the circuit into a ZX-graph
zx.simplify.interior_clifford_simp(g,quiet=False) # if it is not quiet then the amount of reductions in each step is printed
# The following function makes the representation of the graph more compact. 
# It only moves vertices around, and may introduce some identities
# It is recommended to call this function before trying to draw a graph, as otherwise the graph might not be very readable.
g.normalise() 
display(zx.draw(g))
print("Optimized T-count:", zx.tcount(g))

As you can see this routine has decreased the T-count by a bit, but not by a lot.


Part of the reason is that in this circuit there are still a few phaseless nodes 'trapped' in the middle of the circuit. These nodes are however connected to the boundary, so if we unfuse those boundary nodes we can get rid of the interior nodes. This can be done by calling ``zx.simplify.pivot_double_boundary``. As this function changes the graph, we call ``interior_clifford_simp`` afterwards to do whatever new simplifications have become available.

In [ ]:
zx.simplify.pivot_double_boundary(g, quiet=False)
g.normalise()
display(zx.draw(g))
zx.simplify.interior_clifford_simp(g,quiet=False)
g.normalise()
display(zx.draw(g))
print("Optimized T-count:", zx.tcount(g))

Doing this further simplification has decreased the T-count by 2 more!

The method ``clifford_simp`` is a convenience function that calls ``interior_clifford_simp`` and ``pivot_double_boundary`` until no further simplifications are found. It is recommended that you call ``clifford_simp`` instead of the other functions.
In order to further reduce the T-count we have to start using more advanced techniques. Namely the process of *gadgetization*:

In [ ]:
newgadgets, allgadgets = zx.simplify.gadgetize(g)
zx.draw(g)

This method unfuses all T-like phases so that the underlying graph structure becomes available for doing pivot-based simplifications. To do these pivots optimally however, we need to tell the simplifier that it shouldn't touch the new nodes, the 'gadgets', because that would only undo the gadgetization instead of making progress on the graph.

In [ ]:
# This is the match function that tells pivot_simp which edges to consider for simplification
def matchf(e):
    s, t = g.edge_st(e)
    if s in newgadgets or t in newgadgets:
        return False
    else: return True

zx.simplify.pivot_simp(g,matchf=matchf,quiet=False)
g.normalise()
display(zx.draw(g))
print("T-count:", zx.tcount(g))

As you can see, the T-count has not actually been reduced by doing just these pivots, but now the graph has a completely different structure then before, so lets see what happens if we apply ``clifford_simp`` again:

In [ ]:
zx.simplify.clifford_simp(g, quiet=False)
g.normalise()
display(zx.draw(g, figsize=(10,3)))
print("Optimized T-count:", zx.tcount(g))

That has reduced the T-count from 20 down to 12! Note that we are already surpassing the state-of-the-art in this case, as the previously best-known T-count for this circuit was 16.

But we are in fact not done yet. If you look closely at the graph above you will see that a few of the gadgets have exactly the same set of neighbours. Whenever this happens, these phase gadgets can be fused into a single phase gadget by adding the phases together. This procedure is done by ``gadget_simp``:

In [ ]:
zx.simplify.gadget_simp(g, quiet=False)
g.normalise()
display(zx.draw(g))
print("Optimized T-count:", zx.tcount(g))

The T-count has been further reduced by 4!

There is not much we can do now. We do one final round of ``clifford_simp`` to make the graph a bit smaller.

In [ ]:
zx.simplify.clifford_simp(g, quiet=False)
g.normalise()
display(zx.draw(g))

Since it is quite a bit of effort to do all these steps manually, they have been combined into the function ``full_reduce``. This procedure does the following steps:

 1. Run ``clifford_simp``.
 2. Gadgetize all T-like vertices that aren't phase gadgets.
 3. Do ``pivot_simp`` while avoiding gadgets.
 4. Run ``clifford_simp``.
 5. Run ``gadget_simp``, if it finds simplifications go back to step 1, otherwise halt.
 
To demonstrate:

In [ ]:
g = circ.to_graph()
zx.simplify.full_reduce(g,quiet=False)
g.normalise()
zx.draw(g)

The next step is to turn this graph back into a circuit

<a id="optimization-circuits"></a>
# Extracting and optimizing circuits

For extracting circuits out of ZX-graphs there is only a single method in PyZX that you have to call: ``zx.extract.streaming_extract``. This method should always work *when dealing with graphs produced by ``full_reduce``.* There is no guarantee that it can extract circuits from arbitrary ZX-diagrams.

Let's see what this method does when applied to the circuit from the previous section:

In [ ]:
fname = os.path.join('..','circuits','Fast', 'mod5_4_before')
circ = zx.Circuit.load(fname)
print("The original circuit:")
display(zx.draw(circ, figsize=(10,1.5)))

g = circ.to_graph()
zx.simplify.full_reduce(g,quiet=True)
g.normalise()
print("The optimized ZX-diagram:")
display(zx.draw(g))

new_circ = zx.extract.streaming_extract(g)
print("The extracted circuit:")
zx.draw(new_circ, figsize=(10,1.5))

Explaining how ``streaming_extract`` works is out of scope for this notebook for now. If you want to know more, you can contact the authors.

As you can see, the extracted circuit looks quite different from the original circuit, so how can we be sure that they actually represent the same unitary? PyZX allows you to convert ZX-diagrams into the tensors they represent using numpy. In this way we can directly compare the unitaries and see that they are equal:

In [ ]:
# This method checks whether the two given ZX-graphs or circuits have the same tensor representation up to some nonzero scalar
zx.compare_tensors(circ, new_circ)

This method uses ``zx.tensorfy`` under the hood to turn the circuits into tensors. This method is not optimized for memory usage, so it will run out of memory quite quickly (sometimes even for circuits with 9 or 10 qubits). It is only usable for testing small circuits and ZX-diagrams.

``streaming_extract`` does not care about extracting a circuit that is in any way optimal. The amount of 2-qubit gates will therefore often be much higher than it needs to be. If you look at the end of the extracted circuit above you also see that it has some sequences of hadamards in a row that should be cancelled. PyZX offers a circuit optimization method that does this kind of hadamard and 2-qubit gate reduction:

In [ ]:
print(new_circ.to_basic_gates().stats())
optimized_circ = zx.optimize.basic_optimization(new_circ.to_basic_gates())
print(optimized_circ.stats())
zx.draw(optimized_circ, figsize=(10,2))

``basic_optimization`` commutes gates past Hadamards in order to find matching CNOT, CZ and Hadamard gates that can be cancelled. Depending on the circuit it can find significant reductions in the amount of Hadamard gates, which is useful for the next optimization routine.

As you can see, the optimized circuit only contains 2 Hadamard gates, and in fact the leftmost Hadamard gate could be commuted past the CNOTs and CZs to its left so that the entire interior of the circuit is free of Hadamards. Such a Hadamard-free circuit is called a *phase polynomial* circuit, and there are specific techniques for optimizing these types of circuits. PyZX offers a method that finds phase polynomial sub-circuits and runs an optimization routine on them:

In [ ]:
final_circ = zx.optimize.phase_block_optimize(optimized_circ)
final_circ = zx.optimize.basic_optimization(final_circ) # We call this again, as it does some extra processing
print(final_circ.stats())
zx.draw(final_circ)

Let's check once more that this circuit is still equal to the original circuit:

In [ ]:
zx.compare_tensors(circ, final_circ)

And finally lets output this circuit in a format that is usable in other software:

In [ ]:
print(final_circ.to_qasm())